In [41]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.fft import rfft, rfftfreq
import librosa
import librosa.display
import IPython.display as ipd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pickle
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.mixture import GaussianMixture
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
warnings.filterwarnings('ignore')


In [42]:
def mfcc_feature_extractor(audio, sampleRate):
    mfccsFeatures = librosa.feature.mfcc(y=audio, sr=sampleRate, n_mfcc=40)
    mfccsScaledFeatures = np.mean(mfccsFeatures.T, axis=0)

    return mfccsScaledFeatures


In [43]:
# def contrast_feature_extractor(audio, sampleRate):
#     stft = np.abs(librosa.stft(audio))
#     contrast = librosa.feature.spectral_contrast(S=stft, sr=sampleRate)
#     contrastScaled = np.mean(contrast.T, axis=0)

#     return contrastScaled


In [44]:
# def tonnetz_feature_extractor(audio, sampleRate):
#     tonnetz = librosa.feature.tonnetz(
#         y=librosa.effects.harmonic(audio), sr=sampleRate)
#     tonnetzScaled = np.mean(tonnetz.T, axis=0)

#     return tonnetzScaled


In [45]:
# def centroid_feature_extractor(audio, sampleRate):
#     centroid = librosa.feature.spectral_centroid(y=audio, sr=sampleRate)
#     centroidScaled = np.mean(centroid.T, axis=0)

#     return centroidScaled


In [46]:
def features_extractor(file):
    features = []
    # scaler = StandardScaler()
    audio, sampleRate = librosa.load(file, res_type='kaiser_fast')
    mfcc = mfcc_feature_extractor(audio, sampleRate)
    # contrast = contrast_feature_extractor(audio, sampleRate)
    # tonnetz = tonnetz_feature_extractor(audio, sampleRate)
    # centroid = centroid_feature_extractor(audio, sampleRate)
    features.append(mfcc)
                    # , contrast, tonnetz, centroid]

        # , features[0][1], features[0][2], features[0][3])
    return features


In [47]:
def add_features(extractedFeatures,dirPath,label):
    for file in os.listdir(dirPath):
        filePath = dirPath+'/'+file
        finalClassLabels=label
        audio, sampleRate = librosa.load(filePath, res_type='kaiser_fast') 
        mfcc=mfcc_feature_extractor(audio,sampleRate)
        # contrast = contrast_feature_extractor(audio,sampleRate)
        # tonnetz = tonnetz_feature_extractor(audio,sampleRate)
        # centroid = centroid_feature_extractor(audio,sampleRate)
        extractedFeatures.append([mfcc,finalClassLabels])

In [48]:
dict = {"open the door":["C:/Users/EXTRA/DSP_Task3/processing/anwar","C:/Users/EXTRA/DSP_Task3/processing/aya","C:/Users/EXTRA/DSP_Task3/processing/ehab","C:/Users/EXTRA/DSP_Task3/processing/zeyad"],"others":["C:/Users/EXTRA/DSP_Task3/processing/words/Anwar","C:/Users/EXTRA/DSP_Task3/processing/words/Aya","C:/Users/EXTRA/DSP_Task3/processing/words/Ehab","C:/Users/EXTRA/DSP_Task3/processing/words/Zeyad" ]}

In [49]:
extractedFeatures = []
for i in dict.keys():
    for x in dict[i]:
        add_features(extractedFeatures,x,i)

In [50]:
data = pd.DataFrame(extractedFeatures, columns=['mfcc', 'class'])
print(data.shape)
data.describe()


(521, 2)


,mfcc,class
count,521,521
unique,521,2
top,"[-358.41626, 144.1046, -4.3864827, 17.072119, ...",others
freq,1,411


In [51]:
features = data.iloc[:, 0:-1]
target = data['class']
features.tail()


,mfcc
516,"[-403.88452, 100.31129, 12.981465, 7.540295, 4..."
517,"[-394.7898, 95.47343, 18.773857, 13.397423, 1...."
518,"[-395.23132, 95.21313, 24.915659, 11.414965, 6..."
519,"[-407.15182, 80.67283, 25.647009, 16.32321, 7...."
520,"[-425.70676, 78.90517, 27.258017, 16.093987, 1..."


In [52]:
mfcc = np.array(features['mfcc'].tolist())
scaler = StandardScaler()
mfcc = scaler.fit_transform(mfcc)
# contrast = np.array(features['contrast'].tolist())
# tonnetz = np.array(features['tonnetz'].tolist())
# centroid = np.array(features['centroid'].tolist())


In [53]:
features = []
for i in range(len(mfcc)):
    features.append(mfcc[i])
data = pd.DataFrame(features)
features = data.values.tolist()
# for i in range(len(features)):
#     features[i] = np.concatenate((features[i][0],features[i][1],features[i][2],features[i][3]))


In [54]:
encoder = LabelEncoder()
target = encoder.fit_transform(target)


In [55]:
xTrain, xTest, yTrain, yTest = train_test_split(
    features, target, test_size=0.25, random_state=105)


In [56]:
classifier = DecisionTreeClassifier()
classifier.fit(xTrain,yTrain)

DecisionTreeClassifier()

In [57]:
preds = classifier.predict(xTest)


In [58]:
def prediction(pred):
    if pred == 0:
        print("Correct!")
    elif pred == 1:
        print("Wrong password")
  


In [59]:
print(classification_report(yTest,preds))

              precision    recall  f1-score   support

           0       0.94      0.71      0.81        24
           1       0.94      0.99      0.96       107

    accuracy                           0.94       131
   macro avg       0.94      0.85      0.89       131
weighted avg       0.94      0.94      0.94       131



In [60]:
testFeatures = features_extractor(
    "C:/Users/EXTRA/DSP_Task3/processing/ehab/voice1619028.wav")
pred = classifier.predict(testFeatures)
prediction(pred)
testFeatures = features_extractor(
    "C:/Users/EXTRA/DSP_Task3/processing/words/Anwar/voice28110762.wav")
pred = classifier.predict(testFeatures)
prediction(pred)


Correct!
Wrong password


In [61]:
pickle_out = open("model.pkl", "wb")
pickle.dump(classifier, pickle_out)
pickle_out.close()
